Credit for Code: https://www.kaggle.com/code/brsdincer/brain-tumor-prediction-new-data-full-explanation

Importing Libraries and ignoring warnings

Doubts:

Batches -> Data batch and label batch -> what is it?
Ask about image data generator
ask about validation set

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D
from keras import models
from keras import layers
import tensorflow as tf
import os
import os.path
from pathlib import Path
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.optimizers import RMSprop,Adam
import glob
from PIL import Image

# IGNORING UNNECESSARRY WARNINGS

filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

Organization of the Data

Training Set

In [ ]:
#Training Set

No_DataPath = Path("brain-tumor-detection/no/")
Yes_DataPath = Path("brain-tumor-detection/yes/")

No_jpgList = list(No_DataPath.glob(r"*.jpg"))
Yes_jpgList = list(Yes_DataPath.glob(r"*.jpg"))

In [ ]:
#Checking whether list works

print(No_jpgList[0:7])
print("-----------------------"*5)
print(Yes_jpgList[0:7])

In [ ]:
#creating a list with both yes and no pictures

yn_List = []

for i in No_jpgList:
    yn_List.append(i)

for i in Yes_jpgList:
    yn_List.append(i)

print(yn_List[0:5])

In [ ]:
#Creating a list of catagories for better organization

jpgCat = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], yn_List))
print(jpgCat[0:10])



In [ ]:
#Creating Pandas Table

jpgPath_series = pd.Series(yn_List, name="JPG").astype(str)
jpgCat_series = pd.Series(jpgCat, name="Category")

TrainData = pd.concat([jpgPath_series, jpgCat_series], axis = 1)

#Shuffling the data
TrainData = TrainData.sample(frac = 1).reset_index(drop=True)

print(TrainData.head(-1))

Test Set

In [ ]:
#Test Set
testPath = Path("brain-tumor-detection/pred/")
testJPG = list(testPath.glob(r"*.jpg"))

In [ ]:
#Checking if list prints

print(testJPG[0:5])

In [ ]:
#Creating a list of labels for test set

testLabels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],testJPG))
print(testLabels[0:5])

In [21]:
testJPG_Series = pd.Series(testJPG, name = "JPG").astype(str)
testLabels_Series = pd.Series(testLabels, name = "Category")

testData = pd.concat([testJPG_Series, testLabels_Series], axis=1)
print(testData.head())

                                     JPG Category
0   brain-tumor-detection\pred\pred0.jpg     pred
1   brain-tumor-detection\pred\pred1.jpg     pred
2  brain-tumor-detection\pred\pred10.jpg     pred
3  brain-tumor-detection\pred\pred11.jpg     pred
4  brain-tumor-detection\pred\pred12.jpg     pred


Visualizaing in the form of Images

In [ ]:
img = plt.figure(figsize=(10, 10))
plt.imshow(plt.imread(TrainData["JPG"][10]))
plt.title(TrainData["Category"][10])

In [ ]:
fig, axes = plt.subplots(nrows = 5, ncols = 5, figsize=(10,10),subplot_kw ={'xticks':[], 'yticks':[]})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(TrainData["JPG"][i]))
    ax.set_title(TrainData["Category"][i])

plt.tight_layout()
plt.show()

In [19]:
train_data, test_data = train_test_split(TrainData, train_size=0.8, random_state=42)
print(train_data.shape)
print(train_data.head(-1))
print("--------------"*8)
print(test_data.shape)

(2400, 2)
                                      JPG Category
642    brain-tumor-detection\yes\y437.jpg      yes
700    brain-tumor-detection\yes\y925.jpg      yes
226    brain-tumor-detection\yes\y622.jpg      yes
1697  brain-tumor-detection\yes\y1376.jpg      yes
1010   brain-tumor-detection\yes\y176.jpg      yes
...                                   ...      ...
2169   brain-tumor-detection\no\no501.jpg       no
1638   brain-tumor-detection\yes\y169.jpg      yes
1095  brain-tumor-detection\no\no1046.jpg       no
1130   brain-tumor-detection\yes\y145.jpg      yes
1294   brain-tumor-detection\yes\y464.jpg      yes

[2399 rows x 2 columns]
----------------------------------------------------------------------------------------------------------------
(600, 2)


Image Data Generator

In [22]:
from cv2 import batchDistance


Generator = ImageDataGenerator(rescale=1./255, validation_split=0.1)

Train_Set = Generator.flow_from_dataframe(dataframe=train_data,
                                               x_col="JPG",
                                               y_col="Category",
                                               color_mode="grayscale",
                                               class_mode="categorical",
                                               subset="training",
                                               batch_size=20,
                                               target_size=(200,200))

Validation_Set = Generator.flow_from_dataframe(dataframe=train_data,
                                                     x_col="JPG",
                                                     y_col="Category",
                                                     color_mode="grayscale",
                                                     class_mode="categorical",
                                                     subset="validation",
                                                     batch_size=20,
                                                     target_size=(200,200))


Test_Set = Generator.flow_from_dataframe(dataframe=test_data,
                                              x_col="JPG",
                                              y_col="Category",
                                              color_mode="grayscale",
                                              class_mode="categorical",
                                              batch_size=20,
                                              target_size=(200,200))

Found 2160 validated image filenames belonging to 2 classes.
Found 240 validated image filenames belonging to 2 classes.
Found 600 validated image filenames belonging to 2 classes.


In [27]:
for i,j in Train_Set:
    print(i.shape)
    print(j.shape)
    break

(20, 200, 200, 1)
(20, 2)
